In [2]:
import pandas as pd

df=pd.read_csv('data.csv')


In [3]:
df.drop(['PatientId', 'AppointmentID'], axis=1, inplace=True)


In [4]:
#now for female and male i am using 0 and 1
df['Gender'] = df['Gender'].map({'F': 0, 'M': 1})

In [5]:
#i want to find out how many days are there between scheduledday and appointmentday and make it in a column of dayswait=appointment date- scheduled date
df['ScheduledDay'] = pd.to_datetime(df['ScheduledDay'])
df['AppointmentDay'] = pd.to_datetime(df['AppointmentDay'])

In [6]:
#now i want to find out and create new colums dayswait using appointmentday and scheduledday
df['DaysWait'] = (df['AppointmentDay'] - df['ScheduledDay']).dt.days

In [7]:
df.drop(['ScheduledDay', 'AppointmentDay', 'Neighbourhood'], axis=1, inplace=True)

In [8]:
# i want to shift my column of no show to right and dayswait to left
df = df[['Gender', 'Age', 'Scholarship', 'Hipertension', 'Diabetes', 'Alcoholism', 'Handcap', 'SMS_received', 'DaysWait', 'No-show']]

In [9]:
#also want to change no to 1 and yes to 0 in no show
df['No-show'] = df['No-show'].map({'No': 0, 'Yes': 1})

In [10]:
#i want to normalize my age column
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
df['Age_normalized'] = scaler.fit_transform(df[['Age']])


In [11]:
#change in that column why to make another
df.drop(['Age'], axis=1, inplace=True)

In [12]:
#now change name agenormalized to age
df.rename(columns={'Age_normalized': 'Age'}, inplace=True)


In [13]:
#now we have all our dataset ready
df

,Gender,Scholarship,Hipertension,Diabetes,Alcoholism,Handcap,SMS_received,DaysWait,No-show,Age
0,0,0,1,0,0,0,0,-1,0,1.077932
1,1,0,0,0,0,0,0,-1,0,0.818306
2,0,0,0,0,0,0,0,-1,0,1.077932
3,0,0,0,0,0,0,0,-1,0,-1.258708
4,0,0,1,1,0,0,0,-1,0,0.818306
...,...,...,...,...,...,...,...,...,...,...
110522,0,0,0,0,0,0,1,34,0,0.818306
110523,0,0,0,0,0,0,1,34,0,0.601950
110524,0,0,0,0,0,0,1,40,0,-0.696184
110525,0,0,0,0,0,0,1,40,0,0.039425


In [14]:

# Fix invalid ages
df = df[df['Age'] >= 0]

In [15]:
X = df.drop('No-show', axis=1)
y = df['No-show']

In [16]:
from sklearn.model_selection import train_test_split

# Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [17]:
import numpy as np

def initialize_parameters(layer_dims):
    np.random.seed(3)
    parameters = {}
    L = len(layer_dims)  # number of layers including input

    for l in range(1, L):
        parameters['W' + str(l)] = np.random.randn(layer_dims[l], layer_dims[l-1]) * np.sqrt(2. / layer_dims[l-1])
        parameters['b' + str(l)] = np.zeros((layer_dims[l], 1))

    return parameters


In [18]:
def sigmoid(Z):
    return 1 / (1 + np.exp(-Z))

def relu(Z):
    return np.maximum(0, Z)

def linear_forward(A_prev, W, b):
    return np.dot(W, A_prev) + b

In [19]:
def L_layer_forward(X, parameters):
    caches = {}
    A = X
    caches["A0"] = A  # input

    L = len(parameters) // 2  # number of layers

    for l in range(1, L):
        W = parameters['W' + str(l)]
        b = parameters['b' + str(l)]
        Z = linear_forward(A, W, b)
        A = relu(Z)

        caches["Z" + str(l)] = Z
        caches["A" + str(l)] = A

    # Final layer
    W = parameters['W' + str(L)]
    b = parameters['b' + str(L)]
    Z = linear_forward(A, W, b)
    A = sigmoid(Z)

    caches["Z" + str(L)] = Z
    caches["A" + str(L)] = A  # A is Y_hat

    return A, caches


In [20]:
def relu_derivative(Z):
    return Z > 0

def sigmoid(Z):
    return 1 / (1 + np.exp(-Z))

def sigmoid_derivative(A):
    return A * (1 - A)

def update_parameters(parameters, caches, X, Y, learning_rate=0.0001):
    """
    parameters: dictionary of W1, b1, W2, b2, W3, b3
    caches: dictionary of intermediate values Z1, A1, Z2, A2, Z3, A3
    X: input (n_x, m)
    Y: true labels (1, m)
    """

    m = X.shape[1]

    # Retrieve parameters and forward values
    W1, b1 = parameters["W1"], parameters["b1"]
    W2, b2 = parameters["W2"], parameters["b2"]
    W3, b3 = parameters["W3"], parameters["b3"]

    A1, Z1 = caches["A1"], caches["Z1"]
    A2, Z2 = caches["A2"], caches["Z2"]
    A3, Z3 = caches["A3"], caches["Z3"]  # A3 = Y_hat

    # Output layer
    dZ3 = A3 - Y                                # (1, m)
    dW3 = (1/m) * np.dot(dZ3, A2.T)            # (1, 32)
    db3 = (1/m) * np.sum(dZ3, axis=1, keepdims=True)  # (1, 1)

    # Hidden layer 2
    dA2 = np.dot(W3.T, dZ3)                    # (32, m)
    dZ2 = dA2 * relu_derivative(Z2)            # (32, m)
    dW2 = (1/m) * np.dot(dZ2, A1.T)            # (32, 64)
    db2 = (1/m) * np.sum(dZ2, axis=1, keepdims=True)  # (32, 1)

    # Hidden layer 1
    dA1 = np.dot(W2.T, dZ2)                    # (64, m)
    dZ1 = dA1 * relu_derivative(Z1)            # (64, m)
    dW1 = (1/m) * np.dot(dZ1, X.T)             # (64, 9)
    db1 = (1/m) * np.sum(dZ1, axis=1, keepdims=True)  # (64, 1)

    # Update parameters
    parameters["W3"] -= learning_rate * dW3
    parameters["b3"] -= learning_rate * db3

    parameters["W2"] -= learning_rate * dW2
    parameters["b2"] -= learning_rate * db2

    parameters["W1"] -= learning_rate * dW1
    parameters["b1"] -= learning_rate * db1

    return parameters


In [21]:
def compute_loss(Y, Y_hat):
    m = Y.shape[1]
    epsilon = 1e-8  # to prevent log(0)
    loss = -np.sum(Y * np.log(Y_hat + epsilon) + (1 - Y) * np.log(1 - Y_hat + epsilon)) / m
    return loss


In [22]:
parameters = initialize_parameters([9, 64, 32, 1])
epochs = 50
learning_rate = 0.001
Losses = []

for i in range(epochs):
    Loss = []

    for j in range(df.shape[0]):
        # === INPUT ===
        X_sample = df.iloc[j][['Age', 'Scholarship', 'Hipertension', 'Diabetes',
                               'Alcoholism', 'Handcap', 'SMS_received', 'Gender', 'DaysWait']].values.reshape(9, 1)
        y_sample = df.iloc[j]['No-show']  # 0 or 1

        # === FORWARD PASS ===
        y_hat, caches = L_layer_forward(X_sample, parameters)
        y_hat_scalar = y_hat[0][0]

        # === LOSS ===
        loss = -y_sample * np.log(y_hat_scalar + 1e-8) - (1 - y_sample) * np.log(1 - y_hat_scalar + 1e-8)
        Loss.append(loss)

        # === BACKWARD (UPDATE) ===
        parameters = update_parameters(parameters, caches, X_sample, np.array([[y_sample]]), learning_rate)





    print(f"Epoch {i+1} | Loss = {np.mean(Loss):.4f}")


Epoch 1 | Loss = 0.4286
Epoch 2 | Loss = 0.4196
Epoch 3 | Loss = 0.4188
Epoch 4 | Loss = 0.4179
Epoch 5 | Loss = 0.4181
Epoch 6 | Loss = 0.4178
Epoch 7 | Loss = 0.4180
Epoch 8 | Loss = 0.4178
Epoch 9 | Loss = 0.4176
Epoch 10 | Loss = 0.4177
Epoch 11 | Loss = 0.4176
Epoch 12 | Loss = 0.4176
Epoch 13 | Loss = 0.4177
Epoch 14 | Loss = 0.4173
Epoch 15 | Loss = 0.4172
Epoch 16 | Loss = 0.4171
Epoch 17 | Loss = 0.4173
Epoch 18 | Loss = 0.4169
Epoch 19 | Loss = 0.4169
Epoch 20 | Loss = 0.4170
Epoch 21 | Loss = 0.4170
Epoch 22 | Loss = 0.4168
Epoch 23 | Loss = 0.4167
Epoch 24 | Loss = 0.4167
Epoch 25 | Loss = 0.4166
Epoch 26 | Loss = 0.4166
Epoch 27 | Loss = 0.4164
Epoch 28 | Loss = 0.4165
Epoch 29 | Loss = 0.4163
Epoch 30 | Loss = 0.4160
Epoch 31 | Loss = 0.4163
Epoch 32 | Loss = 0.4160
Epoch 33 | Loss = 0.4160
Epoch 34 | Loss = 0.4159
Epoch 35 | Loss = 0.4157
Epoch 36 | Loss = 0.4156
Epoch 37 | Loss = 0.4157
Epoch 38 | Loss = 0.4156
Epoch 39 | Loss = 0.4156
Epoch 40 | Loss = 0.4155
Epoch 41 